In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import pydicom
import numpy as np
import os
import cv2
from cfg import *
# from loader import read_xls


In [1]:
a = 3
id(a)

94132244638592

In [2]:
excel_data = pd.read_excel('./data/PSPF meningioma 20211005.xls', sheet_name="Sheet2")
# excel_data = read_xls()
'''
#ID : [T1 + C , T1 , T2 + C , FLAIR]
non_dict = {ID : [(srs , img) , (srs , img) , (srs , img) , (srs , img)] }
'''
def appendd(id , non_dict , col_name):
    srs, img = (excel_data.loc[index, col_name]).split("/")
    temp = int(srs), int(img)
    non_dict[id].append(temp)

In [3]:
non_dict = {}
pr_dict = {}
for index in range(0 , excel_data.shape[0]) :
    ## naming
    id = excel_data.loc[index, "病歷號 (yellow in WHO grade I file) "]
    pr_status = excel_data.loc[index, "Progression/Recurrence (Yes:1 No:0)"]
    if not pr_status :
        non_dict[id] = []
        appendd(id , non_dict , "T1+C (srs/img)")
        appendd(id, non_dict, "T1 (srs/img)")
        appendd(id, non_dict, "T2 (srs/img)")
        appendd(id, non_dict, "FLAIR (srs/img)")
    else :
        pr_dict[id] = []
        appendd(id, pr_dict, "T1+C (srs/img)")
        appendd(id, pr_dict, "T1 (srs/img)")
        appendd(id, pr_dict, "T2 (srs/img)")
        appendd(id, pr_dict, "FLAIR (srs/img)")
    
print(non_dict)
print("******************************************************")
print(pr_dict)
    



{12261283: [(10, 16), (7, 16), (3, 16), (5, 16)], 13488672: [(72, 14), (6, 14), (5, 15), (4, 15)], 15176128: [(12, 15), (10, 15), (3, 15), (5, 15)], 15525495: [(9, 19), (6, 19), (2, 20), (5, 19)], 15552568: [(1001, 17), (701, 17), (901, 17), (601, 17)], 16962871: [(13, 13), (6, 13), (4, 13), (5, 13)], 17244854: [(27, 17), (5, 18), (10, 18), (6, 18)], 18456622: [(16, 18), (6, 18), (5, 18), (4, 18)], 19315335: [(17, 18), (10, 18), (8, 18), (6, 18)], 19425238: [(12, 17), (8, 17), (3, 17), (7, 17)], 20947834: [(9, 17), (5, 17), (2, 17), (3, 17)], 21105320: [(17, 18), (6, 18), (5, 18), (4, 18)], 21372392: [(11, 14), (7, 15), (4, 15), (6, 14)], 21381973: [(11, 19), (7, 18), (5, 19), (6, 19)], 22296689: [(22, 15), (6, 15), (5, 15), (4, 15)], 25385854: [(16, 21), (9, 21), (5, 21), (4, 21)], 25825974: [(8, 20), (4, 19), (2, 20), (3, 19)], 26073373: [(8, 17), (5, 17), (2, 17), (4, 17)], 26082459: [(9, 19), (5, 20), (2, 20), (4, 19)], 26434694: [(13, 17), (8, 17), (4, 18), (6, 18)], 26658418: [(1

In [28]:
#non pr block 
def convert(ds,store_path , img_type) :
    pixel_array_numpy = ds.pixel_array.astype(float)
    rescaled_img = (np.maximum(pixel_array_numpy, 0) /pixel_array_numpy.max()) * 255
    # image = image.replace('.dcm', '.jpg')

    cv2.imwrite(os.path.join(store_path, f'{img_type}.jpg'), rescaled_img)
        

folder_path = "./data/PSPF meningioma 20210904/non PR"
jpg_folder_path = "./data/PSPF meningioma 20210904/non_PR_jpg"

error_patient = []
for patient_id in os.listdir(folder_path) :
    if patient_id in ['16962871', '14185785']: continue
    patient_path = os.path.join(folder_path , patient_id)
    picture_count = set()
    
    
    store_path = jpg_folder_path + '/' + patient_id #+ "/T1"
    os.makedirs(store_path)

    for n, image in enumerate(os.listdir(patient_path)):
        ds = pydicom.dcmread(os.path.join(patient_path , image))
        #non _dict (every image should be judged 4 times)
        T1 = non_dict[int(patient_id)][0]
        T1c = non_dict[int(patient_id)][1]
        T2 = non_dict[int(patient_id)][2]
        Flair = non_dict[int(patient_id)][3]
        
        if ds.SeriesNumber == T1[0] and ds.InstanceNumber == T1[1]:
            convert(ds, store_path , 'T1')
            picture_count.add('T1')
        if ds.SeriesNumber == T1c[0] and ds.InstanceNumber == T1c[1]:
            convert(ds, store_path, 'T1c')
            picture_count.add('T1c')
        if ds.SeriesNumber == T2[0] and ds.InstanceNumber == T2[1]:
            convert(ds, store_path, 'T2')
            picture_count.add('T2')
        if ds.SeriesNumber == Flair[0] and ds.InstanceNumber == Flair[1]:
            convert(ds, store_path, 'Flair')
            picture_count.add('Flair')
        '''
        pixel_array_numpy = ds.pixel_array.astype(float)
        rescaled_img = (np.maximum(pixel_array_numpy, 0) /
                        pixel_array_numpy.max()) * 255
        image = image.replace('.dcm', '.jpg')
        cv2.imwrite(os.path.join(jpg_folder_path, image), rescaled_img)
        '''
    if len(picture_count) != 4 :
        error_patient.append(patient_id)
    # if patient_id=='16962871':
    #     print(picture_count)
    
if error_patient != [] :
    print(error_patient)

16962871 少一張

In [24]:
#PR block
dcm_folder_path = "./data/PSPF meningioma 20210904/PR"
jpg_folder_path = "./data/PSPF meningioma 20210904/PR_jpg"

error_patient = []
for patient_id in os.listdir(dcm_folder_path):
    patient_path = os.path.join(dcm_folder_path, patient_id)
    picture_count = set()
    
    store_path = jpg_folder_path + '/' + patient_id
    os.makedirs(store_path)
    
    for n, image in enumerate(os.listdir(patient_path)):
        ds = pydicom.dcmread(os.path.join(patient_path, image), force=True)
        
        T1 = pr_dict[int(patient_id)][0]
        T1c = pr_dict[int(patient_id)][1]
        T2 = pr_dict[int(patient_id)][2]
        Flair = pr_dict[int(patient_id)][3]
        try :
            if ds.SeriesNumber == T1[0] and ds.InstanceNumber == T1[1]:
                # store_path = jpg_folder_path + '/' + patients + "/T1"
                # os.makedirs(store_path)
                convert(ds, store_path, 'T1')
                picture_count.add('T1')
            if ds.SeriesNumber == T1c[0] and ds.InstanceNumber == T1c[1]:
                # store_path = jpg_folder_path + '/' + patients + "/T1c"
                # os.makedirs(store_path)
                convert(ds, store_path, 'T1c')
                picture_count.add('T1c')
            if ds.SeriesNumber == T2[0] and ds.InstanceNumber == T2[1]:
                # store_path = jpg_folder_path + '/' + patients + "/T2"
                # os.makedirs(store_path)
                convert(ds, store_path, 'T2')
                picture_count.add('T2')
            if ds.SeriesNumber == Flair[0] and ds.InstanceNumber == Flair[1]:
                # store_path = jpg_folder_path + '/' + patients + "/Flair"
                # os.makedirs(store_path)
                convert(ds, store_path, 'Flair')
                picture_count.add('Flair')
        except :
            error_patient.append(patient_id)
        
    if len(picture_count) != 4:
        error_patient.append(patient_id)

if error_patient != []:
    print(error_patient)


In [9]:
error_patient

[]

patient 14185785 缺excel data
